<a href="https://colab.research.google.com/github/vectara/example-notebooks/blob/main/notebooks/api-examples/5-sub-agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectara Sub-Agents: Building Modular AI Workflows

This notebook demonstrates how to use Vectara's **sub-agents** capability to build modular, specialized AI workflows. Sub-agents allow a parent agent to delegate tasks to specialized child agents, enabling:

- **Context isolation**: Each sub-agent maintains its own conversation history
- **Specialized configuration**: Each sub-agent can have distinct instructions and tools
- **Reusability**: Build once, invoke from any parent agent
- **Parallel execution**: Run multiple sub-agents simultaneously
- **Better performance**: Smaller, focused agents make fewer mistakes

### Why Sub-Agents?

When agents face complex, multi-step tasks, they often run into context window limits or need specialized capabilities. Consider a comprehensive research assistant that needs to:

1. Analyze academic papers for theoretical foundations
2. Search product documentation for implementation details
3. Synthesize findings into actionable recommendations

A single monolithic agent trying to handle all of this might:
- Become confused between different instruction sets
- Consume excessive context with domain-specific guidelines
- Produce lower quality results due to competing priorities

**Sub-agents solve this by delegation**: the parent agent orchestrates, while specialized sub-agents focus on their domains.

We also demonstrate combining sub-agents with a **Lambda Tool** (API Payload Validator) that validates generated API code to catch hallucinations and ensure correctness before presenting to users.

## Getting Started

This notebook assumes you've completed Notebooks 1-4:
- Notebook 1: Created two corpora (ai-research-papers and vectara-docs)
- Notebook 2: Ingested AI research papers and Vectara documentation
- Notebook 3: Queried the data with various techniques
- Notebook 4: Created agents that can search and reason across data

Now we'll create a multi-agent system where specialized sub-agents handle domain-specific tasks.

## Setup

In [1]:
import os
import requests
import json
from datetime import datetime

# Get credentials from environment variables
api_key = os.environ['VECTARA_API_KEY']

# Corpus keys from previous notebooks
research_corpus_key = 'tutorial-ai-research-papers'
docs_corpus_key = 'tutorial-vectara-docs'

# Base API URL
BASE_URL = "https://api.vectara.io/v2"

# Common headers
headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json"
}

print(f"Research Corpus: {research_corpus_key}")
print(f"Docs Corpus: {docs_corpus_key}")

Research Corpus: tutorial-ai-research-papers
Docs Corpus: tutorial-vectara-docs


## Step 1: Create Specialized Sub-Agents

We'll create three specialized agents that will serve as sub-agents:

1. **Research Paper Analyst**: Expert at analyzing academic papers on RAG, embeddings, and retrieval
2. **Documentation Expert**: Expert at finding implementation guidance from Vectara docs
3. **Web Search Expert**: Expert at searching the web for current information and news

Each agent has focused instructions and tools optimized for its domain.

### Sub-Agent 1: Research Paper Analyst

In [2]:
# Helper function to delete and create agent
def delete_and_create_agent(agent_config, agent_name):
    """Delete agent if it exists, then create a new one."""
    # Check if agent already exists and delete it
    list_response = requests.get(f"{BASE_URL}/agents", headers=headers)

    if list_response.status_code == 200:
        agents = list_response.json().get('agents', [])
        for agent in agents:
            if agent.get('name') == agent_name:
                existing_key = agent['key']
                print(f"Deleting existing agent '{agent_name}' ({existing_key})")
                delete_response = requests.delete(f"{BASE_URL}/agents/{existing_key}", headers=headers)
                if delete_response.status_code == 204:
                    print(f"Deleted agent: {existing_key}")
                else:
                    print(f"Error deleting {existing_key}: {delete_response.text}")
                break

    # Create new agent
    response = requests.post(f"{BASE_URL}/agents", headers=headers, json=agent_config)

    if response.status_code == 201:
        agent_data = response.json()
        print(f"Created agent '{agent_name}'")
        print(f"Agent Key: {agent_data['key']}")
        return agent_data['key']
    else:
        print(f"Error creating agent: {response.status_code}")
        print(f"{response.text}")
        return None

In [3]:
# Create Research Paper Analyst sub-agent
reranker_config = {
    "type": "chain",
    "rerankers": [
        {
            "type": "customer_reranker",
            "reranker_id": "rnk_272725719", 
            "limit": 25,
        },
        {
            "type": "mmr",
            "diversity_bias": 0.05
        }
    ],
}

research_analyst_config = {
    "name": "Research Paper Analyst",
    "description": "Specialized agent for analyzing academic research papers on RAG, embeddings, and retrieval techniques",
    "model": {"name": "gpt-4o"},
    "first_step": {
        "type": "conversational",
        "instructions": [
            {
                "type": "inline",
                "name": "research_analyst_instructions",
                "template": """You are an expert academic research analyst specializing in AI, machine learning, and natural language processing.

Your expertise includes:
- Retrieval Augmented Generation (RAG) architectures
- Dense and sparse retrieval methods
- Embedding models and vector representations
- Transformer architectures and attention mechanisms
- Information retrieval benchmarks and evaluation metrics

When analyzing research papers:
1. Identify the key contributions and novel techniques
2. Explain technical concepts clearly with examples
3. Highlight practical implications and limitations
4. Compare with related work when relevant
5. Provide citations to the source papers

Always use tools to retrieve relevant content to answer user queries.
Always ground your response in the retrieved content. 
If you cannot answer the user question from the retrieved content from tools, just say "I don't know"

IMPORTANT: When responding, provide a complete, self-contained summary that includes all relevant findings."""
            }
        ],
        "output_parser": {"type": "default"}
    },
    "tool_configurations": {
        "research_search": {
            "type": "corpora_search",
            "query_configuration": {
                "search": {
                    "corpora": [{"corpus_key": research_corpus_key}],
                    "limit": 100,
                    "context_configuration": {
                        "sentences_before": 2,
                        "sentences_after": 2
                    },
                    "reranker": reranker_config,                   
                },
                "save_history": True,
            }
        }
    }
}

research_analyst_key = delete_and_create_agent(research_analyst_config, "Research Paper Analyst")

Created agent 'Research Paper Analyst'
Agent Key: agt_research_paper_analyst_957d


### Sub-Agent 2: Vectara Documentation Expert

In [4]:
# Create Documentation Expert sub-agent

docs_expert_config = {
    "name": "Documentation Expert",
    "description": "Specialized agent for finding implementation guidance and best practices from Vectara documentation",
    "model": {"name": "gpt-4o"},
    "first_step": {
        "type": "conversational",
        "instructions": [
            {
                "type": "inline",
                "name": "docs_expert_instructions",
                "template": """You are a Vectara platform expert who helps developers implement AI solutions.

Your expertise includes:
- Vectara API integration (indexing, querying, agents)
- Corpus management and configuration
- Search optimization (hybrid search, reranking, filters)
- RAG implementation best practices
- SDK usage and code examples

When providing guidance:
1. Give specific, actionable implementation steps using the API.
2. Include relevant API endpoints and parameters
3. Your examples should show how to use the API, not using Vectara SDK.
4. Highlight configuration options and trade-offs
5. Point to relevant documentation sections

Always use tools to retrieve relevant content to answer user queries.
Always ground your response in the retrieved content. 
If you cannot answer the user question from the retrieved content from tools, just say "I don't know"

When responding, provide a complete, self-contained answer with all implementation details."""
            }
        ],
        "output_parser": {"type": "default"}
    },
    "tool_configurations": {
        "docs_search": {
            "type": "corpora_search",
            "query_configuration": {
                "search": {
                    "corpora": [{"corpus_key": docs_corpus_key}],
                    "limit": 100,
                    "context_configuration": {
                        "sentences_before": 2,
                        "sentences_after": 2
                    },
                    "reranker": reranker_config,                   
                },
                "save_history": True,
            }
        }
    }
}

docs_expert_key = delete_and_create_agent(docs_expert_config, "Documentation Expert")

Created agent 'Documentation Expert'
Agent Key: agt_documentation_expert_1571


### Sub-Agent 3: Web Search Expert

In [5]:
# Create Web Search Expert sub-agent

web_search_expert_config = {
    "name": "Web Search Expert",
    "description": "Specialized agent for searching the web for current information, news, and general knowledge",
    "model": {"name": "gpt-4o"},
    "first_step": {
        "type": "conversational",
        "instructions": [
            {
                "type": "inline",
                "name": "web_search_expert_instructions",
                "template": """You are a web search expert who helps find current and relevant information from the internet.

Your expertise includes:
- Finding up-to-date information on any topic
- Researching current events and news
- Locating authoritative sources and references
- Comparing information across multiple sources
- Fact-checking and verification

When searching and responding:
1. Use web search to find relevant, current information
2. Prioritize authoritative and credible sources
3. Provide context about when information was published
4. Cite your sources with URLs when available
5. Synthesize information from multiple sources when appropriate

Always use the web search tool to find information.
If you cannot find relevant information, say so clearly.

IMPORTANT: When responding, provide a complete, well-sourced answer with citations."""
            }
        ],
        "output_parser": {"type": "default"}
    },
    "tool_configurations": {
        "web_search": {
            "type": "web_search"
        }
    }
}

web_search_expert_key = delete_and_create_agent(web_search_expert_config, "Web Search Expert")

Created agent 'Web Search Expert'
Agent Key: agt_web_search_expert_b15c


## Step 2: Create an API Validator Lambda Tool

Before creating the orchestrator, we'll add a **Lambda Tool** that validates Vectara API payloads. This tool is *essential* for answering implementation questions because:

- LLMs may hallucinate field names (e.g., `corpus_id` instead of `corpus_key`)
- Parameter ranges need validation (e.g., `lexical_interpolation` must be 0-1)
- Required fields must be present for the code to actually work
- The orchestrator can't claim code is "working" without validating it first

Unlike optional analysis tools that LLMs may skip, this validator is essential—the orchestrator must use it before presenting API code examples to ensure correctness.

In [6]:
# Helper function to manage lambda tools
def delete_and_create_tool(tool_config, tool_name):
    """Delete tool if it exists, then create a new one."""
    list_response = requests.get(f"{BASE_URL}/tools", headers=headers)
    
    if list_response.status_code == 200:
        tools = list_response.json().get('tools', [])
        for tool in tools:
            if tool.get('name') == tool_name:
                existing_id = tool['id']
                print(f"Deleting existing tool '{tool_name}' ({existing_id})")
                delete_response = requests.delete(f"{BASE_URL}/tools/{existing_id}", headers=headers)
                if delete_response.status_code == 204:
                    print(f"Deleted tool: {existing_id}")
                break
    
    response = requests.post(f"{BASE_URL}/tools", headers=headers, json=tool_config)
    
    if response.status_code == 201:
        tool_data = response.json()
        print(f"Created tool '{tool_name}'")
        print(f"Tool ID: {tool_data['id']}")
        return tool_data['id']
    else:
        print(f"Error creating tool: {response.status_code}")
        print(response.text)
        return None

In [7]:
# Create the Vectara API Payload Validator lambda tool
api_validator_code = '''
import json

def process(
    endpoint: str,
    payload: str,
    method: str = "POST"
) -> dict:
    """
    Validate a Vectara API request payload.
    
    Args:
        endpoint: API endpoint (query, index, corpus, agents)
        payload: JSON string of the request body
        method: HTTP method (POST, GET, etc.)
    
    Returns:
        Validation result with errors, warnings, and corrected payload
    """
    errors = []
    warnings = []
    
    # Parse the payload
    try:
        data = json.loads(payload)
    except json.JSONDecodeError as e:
        return {
            "valid": False,
            "endpoint": endpoint,
            "errors": [f"Invalid JSON: {str(e)}"],
            "warnings": [],
            "corrected_payload": None
        }
    
    corrected = dict(data)

    # Validate API version - must use v2, not v1
    if 'vectara.io' in endpoint:
        if '/v1/' in endpoint or '/v1' in endpoint:
            errors.append("Invalid API version: v1 is deprecated. Use https://api.vectara.io/v2 endpoints.")

    # Validation rules by endpoint
    if endpoint == "query":
        # Required fields
        if "query" not in data:
            errors.append("Missing required field: 'query'")
            corrected["query"] = "<YOUR_QUERY>"
        
        if "search" not in data:
            errors.append("Missing required field: 'search'")
            corrected["search"] = {"corpora": [{"corpus_key": "<YOUR_CORPUS_KEY>"}]}
        else:
            search = data.get("search", {})
            if "corpora" not in search:
                errors.append("Missing required field: 'search.corpora'")
            else:
                for i, corpus in enumerate(search.get("corpora", [])):
                    if "corpus_key" not in corpus:
                        errors.append(f"Missing corpus_key in corpora[{i}]")
                    
                    # Validate lexical_interpolation range
                    if "lexical_interpolation" in corpus:
                        li = corpus["lexical_interpolation"]
                        if not isinstance(li, (int, float)) or li < 0 or li > 1:
                            errors.append(f"lexical_interpolation must be 0-1, got: {li}")
                            corrected["search"]["corpora"][i]["lexical_interpolation"] = 0.025
            
            # Validate limit
            if "limit" in search:
                limit = search["limit"]
                if not isinstance(limit, int) or limit < 1:
                    warnings.append(f"limit should be positive integer, got: {limit}")
        
        # Check for common hallucinations
        hallucinated_fields = ["corpus_id", "customer_id", "num_results", "query_text"]
        for field in hallucinated_fields:
            if field in data:
                errors.append(f"Invalid field '{field}' - this is a hallucination. Did you mean: " +
                            {"corpus_id": "search.corpora[].corpus_key",
                             "customer_id": "(not needed with API key)",
                             "num_results": "search.limit",
                             "query_text": "query"}.get(field, "check docs"))
    
    elif endpoint == "index":
        if "document_id" not in data and "id" not in data:
            errors.append("Missing required field: 'document_id' or 'id'")
        
        has_content = "parts" in data or "document_parts" in data or "content" in data
        if not has_content:
            errors.append("Missing content: need 'parts', 'document_parts', or 'content'")
        
        # Check for hallucinations
        if "text" in data and "parts" not in data:
            warnings.append("'text' alone is not valid - use 'parts' array with 'text' inside")
    
    elif endpoint == "agents":
        if "name" not in data:
            errors.append("Missing required field: 'name'")
        if "model" not in data:
            errors.append("Missing required field: 'model'")
        if "first_step" not in data:
            errors.append("Missing required field: 'first_step'")
    
    elif endpoint == "corpus":
        if "key" not in data and "name" not in data:
            warnings.append("Consider providing 'key' or 'name' for the corpus")
    
    return {
        "valid": len(errors) == 0,
        "endpoint": endpoint,
        "method": method,
        "errors": errors,
        "warnings": warnings,
        "corrected_payload": corrected if errors else data
    }
'''

api_validator_config = {
    "type": "lambda",
    "language": "python",
    "name": "vectara_api_validator",
    "title": "Vectara API Payload Validator",
    "description": "Validate Vectara API request payloads before presenting to users. Checks required fields, validates parameter ranges (e.g., lexical_interpolation 0-1), and catches common LLM hallucinations (wrong field names). Returns validation status, errors, and corrected payload. ALWAYS use this before showing API code examples.",
    "code": api_validator_code
}

api_validator_id = delete_and_create_tool(api_validator_config, "vectara_api_validator")

Created tool 'vectara_api_validator'
Tool ID: tol_2306


## Step 3: Create the Parent Orchestrator Agent

Now we'll create a parent agent that can delegate to the sub-agents and use the API validator. The parent agent:
- Analyzes user requests to determine which sub-agent(s) to invoke
- Delegates domain-specific tasks to the appropriate sub-agent
- **Uses the API validator** to ensure generated code is correct before presenting it back to the user.
- Synthesizes responses from multiple sub-agents into a cohesive answer

### Sub-Agent Tool Configuration

Sub-agents are configured as tools using the `sub_agent` type:

```json
{
  "type": "sub_agent",
  "description_template": "Description the LLM sees when deciding to use this tool",
  "sub_agent_configuration": {
    "agent_key": "the_sub_agent_key",
    "session_mode": "ephemeral"
  }
}
```

### Session Modes

When specifying in the parent agent how sub-agents are to be used, you need to define the "session mode":
- **`persistent`**: Always reuse the same session, accumulating knowledge across invocations
- **`ephemeral`**: Create a fresh session every time, ensuring no state leakage
- **`llm_controlled`**: The LLM decides whether to resume an existing session or create a new one


In [8]:
# Create the Orchestrator Agent with sub-agent tools and API validator
orchestrator_config = {
    "name": "AI Research Orchestrator",
    "description": "Orchestrator agent that delegates to specialized sub-agents for comprehensive AI research assistance",
    "model": {"name": "gpt-4o"},
    "first_step": {
        "type": "conversational",
        "instructions": [
            {
                "type": "inline",
                "name": "orchestrator_instructions",
                "template": """You are an AI research orchestrator that helps users understand and implement AI technologies.

Your workflow:
1. Analyze the user's question to determine what expertise is needed.
2. Use appropriate sub-agent(s) to gather information:
   - research_analyst: for academic/theoretical content
   - docs_expert: for implementation guidance
   - web_search_expert: for current news/trends
3. When your response includes Vectara API code examples or implementation details:
   - Generate the JSON payload
   - ALWAYS validate it using the api_validator tool before presenting
   - If validation fails, fix the errors and validate again
   - Only present validated, working code to the user
4. Synthesize all responses into a comprehensive answer.

IMPORTANT: 
- Never show API code without validating it first. Users trust that code examples will work.
- When responding, provide a complete, well-sourced answer with citations.
"""
            }
        ],
        "output_parser": {"type": "default"}
    },
    "tool_configurations": {
        "research_analyst": {
            "type": "sub_agent",
            "description_template": "Research academic research papers for theoretical foundations and research findings.",
            "sub_agent_configuration": {
                "agent_key": research_analyst_key,
                "session_mode": "ephemeral"
            }
        },
        "docs_expert": {
            "type": "sub_agent",
            "description_template": "Research Vectara documentation for API usage, code examples, configuration, and best practices.",
            "sub_agent_configuration": {
                "agent_key": docs_expert_key,
                "session_mode": "ephemeral"
            }
        },
        "web_search_expert": {
            "type": "sub_agent",
            "description_template": "Search the web for up-to-date information, news, and current trends.",
            "sub_agent_configuration": {
                "agent_key": web_search_expert_key,
                "session_mode": "ephemeral"
            }
        },
        "api_validator": {
            "type": "lambda",
            "tool_id": api_validator_id
        }
    }
}

orchestrator_key = delete_and_create_agent(orchestrator_config, "AI Research Orchestrator")

Created agent 'AI Research Orchestrator'
Agent Key: agt_ai_research_orchestrator_85ef


## Step 4: Test the Multi-Agent Workflow

Now let's test the orchestrator with different types of questions to see how it delegates to sub-agents and uses the API validator.

In [9]:
# Helper function to send messages and display responses
def chat_with_agent(agent_key, session_key, message, show_events=False):
    """Send a message to an agent and return the response."""
    message_data = {
        "messages": [
            {
                "type": "text",
                "content": message
            }
        ],
        "stream_response": False
    }
    
    url = f"{BASE_URL}/agents/{agent_key}/sessions/{session_key}/events"
    response = requests.post(url, headers=headers, json=message_data)
    
    if response.status_code == 201:
        event_data = response.json()
        
        if show_events:
            print("\n------ Agent Events ------")
            for event in event_data.get('events', []):
                event_type = event.get('type', 'INVALID')
                print(f"Event: {event_type}")
                if event_type == 'tool_input':
                    tool_name = event.get('tool_configuration_name', 'N/A')
                    print(f"  Tool: {tool_name}")
                    tool_input = event.get("tool_input", {})
                    if tool_input.get("message"):
                        print(f"  Input: {tool_input['message']}...")
                    # Show API validator inputs
                    if tool_input.get("endpoint"):
                        print(f"  Validating endpoint: {tool_input['endpoint']}")
                if event_type == 'tool_output':
                    tool_name = event.get('tool_configuration_name', 'N/A')
                    print(f"  Tool: {tool_name}")
                    tool_output = event.get("tool_output", {})
                    # Show sub-agent response preview
                    if tool_output.get("sub_agent_response"):
                        print(f"  Response (200 chars): {tool_output['sub_agent_response'][:200]}...")
                    # Show API validator output
                    if tool_output.get("lambda_response"):
                        lambda_resp = tool_output["lambda_response"]
                        if "valid" in lambda_resp:
                            valid = lambda_resp["valid"]
                            print(f"  Validation Result: {'VALID' if valid else 'INVALID'}")
                            if lambda_resp.get("errors"):
                                print(f"    Errors: {lambda_resp['errors']}")
                            if lambda_resp.get("warnings"):
                                print(f"    Warnings: {lambda_resp['warnings']}")
            print("-"*25)
            print("\n")
        
        # Extract agent output
        for event in event_data.get('events', []):
            if event.get('type') == 'agent_output':
                return event.get('content', 'No content')
        
        return "No agent output found"
    else:
        return f"Error: {response.status_code} - {response.text}"

In [10]:
# Create a session for the orchestrator
session_name = f"Orchestrator Session {datetime.now().strftime('%Y%m%d-%H%M%S')}"
session_config = {
    "name": session_name,
    "metadata": {
        "purpose": "sub_agent_demo"
    }
}

response = requests.post(
    f"{BASE_URL}/agents/{orchestrator_key}/sessions",
    headers=headers,
    json=session_config
)

if response.status_code == 201:
    session_data = response.json()
    orchestrator_session_key = session_data["key"]
    print(f"Session Created: {orchestrator_session_key}")
else:
    print(f"Error: {response.text}")

Session Created: ase_orchestrator_session_20251215-163417_9b8c


### Test 1: Research-Focused Question

This question should primarily use the research_analyst sub-agent:

In [11]:
query = "What are the key innovations in RAG?"
print(f"User: {query}")
print("\n" + "="*80)

response = chat_with_agent(
    orchestrator_key,
    orchestrator_session_key,
    query,
    show_events=True
)

print(f"Agent Response:\n\n{response}")

User: What are the key innovations in RAG?


------ Agent Events ------
Event: input_message
Event: tool_input
  Tool: research_analyst
  Input: Research the latest academic findings and theoretical advancements in Retrieval-Augmented Generation (RAG) models. Focus on key innovations and improvements in this field....
Event: tool_input
  Tool: web_search_expert
  Input: Find the latest news and articles about innovations and trends in Retrieval-Augmented Generation (RAG) models. Focus on recent developments and practical applications....
Event: tool_output
  Tool: research_analyst
  Response (200 chars): Recent advancements in Retrieval-Augmented Generation (RAG) models focus on addressing challenges in combining parametric and non-parametric memory for more effective language generation. Here are the...
Event: tool_output
  Tool: web_search_expert
  Response (200 chars): Recent advancements in Retrieval-Augmented Generation (RAG) models, as of 2023, highlight several key innovations a

### Test 2: Implementation Question with API Validation

This question asks for working code, which should trigger the API validator to ensure correctness:

In [12]:
query = "Give me working code to search a corpus for AI research papers using hybrid search with Vectara's API."
print(f"User: {query}")
print("\n" + "="*80)

response = chat_with_agent(
    orchestrator_key,
    orchestrator_session_key,
    query,
    show_events=True
)

print(f"Agent Response:\n\n{response}")

User: Give me working code to search a corpus for AI research papers using hybrid search with Vectara's API.


------ Agent Events ------
Event: input_message
Event: tool_input
  Tool: docs_expert
  Input: Provide a code example for using Vectara's API to perform a hybrid search on a corpus for AI research papers. The code should include the necessary steps and configurations for authenticating and querying the API....
Event: tool_output
  Tool: docs_expert
  Response (200 chars): To perform a hybrid search using Vectara's API for querying a corpus containing AI research papers, follow these steps. This will include setting up the necessary configuration, authenticating, and ma...
Event: tool_input
  Tool: api_validator
  Validating endpoint: https://api.vectara.io/v2/queries
Event: tool_output
  Tool: api_validator
Event: agent_output
-------------------------


Agent Response:

Here is the validated code to perform a hybrid search using Vectara's API to query a corpus for AI research

### Test 3: Current Information Question (Web Search)

This question asks about recent developments that wouldn't be in our indexed research papers or documentation, demonstrating the value of the web search sub-agent:

In [13]:
query = "What are the latest developments in RAG technology in 2025? Are there any new techniques or frameworks that have emerged recently?"
print(f"User: {query}")
print("\n" + "="*80)

response = chat_with_agent(
    orchestrator_key,
    orchestrator_session_key,
    query,
    show_events=True
)

print(f"Agent Response:\n\n{response}")

User: What are the latest developments in RAG technology in 2025? Are there any new techniques or frameworks that have emerged recently?


------ Agent Events ------
Event: input_message
Event: tool_input
  Tool: web_search_expert
  Input: Search for the latest developments and innovations in Retrieval-Augmented Generation (RAG) technologies in 2025. Look for new techniques, frameworks, and any emerging trends or industry news relevant to this field....
Event: tool_output
  Tool: web_search_expert
  Response (200 chars): In 2025, Retrieval-Augmented Generation (RAG) continues to advance with several innovative techniques and trends that enhance AI's ability to integrate external data for more dynamic performance:

1. ...
Event: agent_output
-------------------------


Agent Response:

In 2025, there have been several notable advancements and innovations in Retrieval-Augmented Generation (RAG) technology that aim to enhance AI capabilities through improved data integration and interacti

## Cleanup (Optional)

If you want to delete the agents created in this notebook:

In [14]:
# Delete agents
agents_to_delete = [
    orchestrator_key,
    research_analyst_key,
    docs_expert_key,
    web_search_expert_key
]

for agent_key in agents_to_delete:
    if agent_key:
        response = requests.delete(f"{BASE_URL}/agents/{agent_key}", headers=headers)
        if response.status_code == 204:
            print(f"Deleted agent: {agent_key}")
        else:
            print(f"Error deleting {agent_key}: {response.text}")

# Delete the lambda tool
if api_validator_id:
    response = requests.delete(f"{BASE_URL}/tools/{api_validator_id}", headers=headers)
    if response.status_code == 204:
        print(f"Deleted tool: {api_validator_id}")
    else:
        print(f"Error deleting tool: {response.text}")

Deleted agent: agt_ai_research_orchestrator_85ef
Deleted agent: agt_research_paper_analyst_957d
Deleted agent: agt_documentation_expert_1571
Deleted agent: agt_web_search_expert_b15c
Deleted tool: tol_2306
